In [1]:
## Reading libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import KMeans
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
import warnings

In [2]:
train  = pd.read_csv('Train.csv')
test  = pd.read_csv('Test.csv')
sampleSubmission = pd.read_csv('SampleSubmission.csv')

In [3]:
# building my target veriable 
target = train['target']

In [5]:
#Removing some noise to our data 
train = train.drop(columns=['Place_ID X Date','Date','Place_ID','target', 'target_min', 'target_max', 'target_variance', 'target_count', 'L3_NO2_sensor_azimuth_angle', 'L3_NO2_sensor_zenith_angle', 
                            'L3_NO2_solar_azimuth_angle', 'L3_NO2_solar_azimuth_angle', 'L3_O3_sensor_azimuth_angle', 'L3_O3_sensor_zenith_angle',
                            'L3_O3_solar_azimuth_angle','L3_O3_solar_zenith_angle','L3_CO_sensor_azimuth_angle', 'L3_CO_sensor_zenith_angle', 'L3_CO_solar_azimuth_angle',
                            'L3_CO_solar_zenith_angle', 'L3_HCHO_sensor_azimuth_angle', 'L3_HCHO_sensor_zenith_angle', 'L3_HCHO_solar_azimuth_angle','L3_HCHO_solar_zenith_angle',
                            'L3_CLOUD_sensor_azimuth_angle', 'L3_CLOUD_sensor_zenith_angle', 'L3_CLOUD_solar_azimuth_angle', 'L3_CLOUD_solar_zenith_angle', 'L3_AER_AI_sensor_azimuth_angle',
                           'L3_AER_AI_sensor_zenith_angle', 'L3_AER_AI_solar_azimuth_angle', 'L3_AER_AI_solar_zenith_angle', 'L3_SO2_sensor_azimuth_angle', 'L3_SO2_sensor_zenith_angle',
                           'L3_SO2_solar_azimuth_angle','L3_SO2_solar_zenith_angle', 'L3_CH4_CH4_column_volume_mixing_ratio_dry_air','L3_CH4_aerosol_height','L3_CH4_aerosol_optical_depth',
                           'L3_CH4_sensor_azimuth_angle', 'L3_CH4_sensor_zenith_angle', 'L3_CH4_solar_azimuth_angle', 'L3_CH4_solar_zenith_angle','L3_NO2_solar_zenith_angle'], axis=1)


In [7]:
test = test.drop(columns=['Place_ID X Date','Date','Place_ID','L3_NO2_sensor_azimuth_angle', 'L3_NO2_sensor_zenith_angle', 
                            'L3_NO2_solar_azimuth_angle', 'L3_NO2_solar_zenith_angle','L3_NO2_solar_azimuth_angle', 'L3_O3_sensor_azimuth_angle', 'L3_O3_sensor_zenith_angle',
                            'L3_O3_solar_azimuth_angle','L3_O3_solar_zenith_angle','L3_CO_sensor_azimuth_angle', 'L3_CO_sensor_zenith_angle', 'L3_CO_solar_azimuth_angle',
                            'L3_CO_solar_zenith_angle', 'L3_HCHO_sensor_azimuth_angle', 'L3_HCHO_sensor_zenith_angle', 'L3_HCHO_solar_azimuth_angle','L3_HCHO_solar_zenith_angle',
                            'L3_CLOUD_sensor_azimuth_angle', 'L3_CLOUD_sensor_zenith_angle', 'L3_CLOUD_solar_azimuth_angle', 'L3_CLOUD_solar_zenith_angle', 'L3_AER_AI_sensor_azimuth_angle',
                           'L3_AER_AI_sensor_zenith_angle', 'L3_AER_AI_solar_azimuth_angle', 'L3_AER_AI_solar_zenith_angle', 'L3_SO2_sensor_azimuth_angle', 'L3_SO2_sensor_zenith_angle',
                           'L3_SO2_solar_azimuth_angle','L3_SO2_solar_zenith_angle', 'L3_CH4_CH4_column_volume_mixing_ratio_dry_air','L3_CH4_aerosol_height','L3_CH4_aerosol_optical_depth',
                           'L3_CH4_sensor_azimuth_angle', 'L3_CH4_sensor_zenith_angle', 'L3_CH4_solar_azimuth_angle', 'L3_CH4_solar_zenith_angle','L3_NO2_solar_zenith_angle'], axis=1)


In [8]:
train.isna().sum().T

precipitable_water_entire_atmosphere                      0
relative_humidity_2m_above_ground                         0
specific_humidity_2m_above_ground                         0
temperature_2m_above_ground                               0
u_component_of_wind_10m_above_ground                      0
v_component_of_wind_10m_above_ground                      0
L3_NO2_NO2_column_number_density                       2189
L3_NO2_NO2_slant_column_number_density                 2189
L3_NO2_absorbing_aerosol_index                         2189
L3_NO2_cloud_fraction                                  2189
L3_NO2_sensor_altitude                                 2189
L3_NO2_stratospheric_NO2_column_number_density         2190
L3_NO2_tropopause_pressure                             2190
L3_NO2_tropospheric_NO2_column_number_density          8669
L3_O3_O3_column_number_density                          283
L3_O3_O3_effective_temperature                          283
L3_O3_cloud_fraction                    

In [9]:
train.shape, test.shape

((30557, 39), (16136, 39))

In [11]:
#train.isna().sum()
#test.isna().sum()

In [12]:
train = train.fillna(train.median())
test = test.fillna(test.median())

In [13]:
#train.isnull().sum()
#test.isnull().sum()

In [14]:
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold,StratifiedKFold, TimeSeriesSplit

testsplit_store=[]
test_store=[]
fold=KFold(n_splits=20, shuffle=True)
i=1
for train_index, test_index in fold.split(train,target):
    X_train, X_test = train.iloc[train_index], train.iloc[test_index]
    y_train, y_test = target.iloc[train_index], target.iloc[test_index]
    cat = CatBoostRegressor(n_estimators=10000, eval_metric='RMSE',learning_rate=0.2, l2_leaf_reg=4)
    cat.fit(X_train,y_train, early_stopping_rounds=100,verbose=100)
    predict = cat.predict(X_test)
    print("err: ",np.sqrt(mean_squared_error(y_test,predict)))
    testsplit_store.append(np.sqrt(mean_squared_error(y_test,predict)))
    pred = cat.predict(test)
    test_store.append(pred)

0:	learn: 44.1316302	total: 292ms	remaining: 48m 37s
100:	learn: 28.9223653	total: 2.31s	remaining: 3m 46s
200:	learn: 25.6887837	total: 4.39s	remaining: 3m 33s
300:	learn: 23.6000019	total: 6.63s	remaining: 3m 33s
400:	learn: 21.9640391	total: 8.89s	remaining: 3m 32s
500:	learn: 20.6250207	total: 11s	remaining: 3m 29s
600:	learn: 19.5432268	total: 13s	remaining: 3m 23s
700:	learn: 18.5273351	total: 14.9s	remaining: 3m 18s
800:	learn: 17.6622532	total: 16.9s	remaining: 3m 14s
900:	learn: 16.8459726	total: 18.9s	remaining: 3m 11s
1000:	learn: 16.1600414	total: 21.1s	remaining: 3m 9s
1100:	learn: 15.5009876	total: 23.6s	remaining: 3m 11s
1200:	learn: 14.8803857	total: 26s	remaining: 3m 10s
1300:	learn: 14.2819121	total: 28.2s	remaining: 3m 8s
1400:	learn: 13.7734246	total: 30.2s	remaining: 3m 5s
1500:	learn: 13.2664030	total: 32.2s	remaining: 3m 2s
1600:	learn: 12.7819838	total: 34.3s	remaining: 3m
1700:	learn: 12.3637757	total: 36.4s	remaining: 2m 57s
1800:	learn: 11.9283178	total: 38.7

5100:	learn: 4.5415817	total: 1m 46s	remaining: 1m 41s
5200:	learn: 4.4219226	total: 1m 48s	remaining: 1m 39s
5300:	learn: 4.3083198	total: 1m 50s	remaining: 1m 37s
5400:	learn: 4.1944852	total: 1m 52s	remaining: 1m 35s
5500:	learn: 4.0836171	total: 1m 54s	remaining: 1m 33s
5600:	learn: 3.9777296	total: 1m 56s	remaining: 1m 31s
5700:	learn: 3.8738302	total: 1m 58s	remaining: 1m 29s
5800:	learn: 3.7734148	total: 2m	remaining: 1m 27s
5900:	learn: 3.6791207	total: 2m 2s	remaining: 1m 25s
6000:	learn: 3.5864844	total: 2m 4s	remaining: 1m 22s
6100:	learn: 3.5018271	total: 2m 6s	remaining: 1m 21s
6200:	learn: 3.4110115	total: 2m 8s	remaining: 1m 18s
6300:	learn: 3.3263834	total: 2m 10s	remaining: 1m 16s
6400:	learn: 3.2432505	total: 2m 12s	remaining: 1m 14s
6500:	learn: 3.1660442	total: 2m 14s	remaining: 1m 12s
6600:	learn: 3.0937061	total: 2m 16s	remaining: 1m 10s
6700:	learn: 3.0187052	total: 2m 18s	remaining: 1m 8s
6800:	learn: 2.9465115	total: 2m 20s	remaining: 1m 6s
6900:	learn: 2.87372

100:	learn: 28.9511568	total: 1.95s	remaining: 3m 11s
200:	learn: 25.9581277	total: 4.18s	remaining: 3m 23s
300:	learn: 23.7709969	total: 6.36s	remaining: 3m 25s
400:	learn: 22.2101283	total: 8.43s	remaining: 3m 21s
500:	learn: 20.8212595	total: 10.5s	remaining: 3m 19s
600:	learn: 19.6927996	total: 12.5s	remaining: 3m 15s
700:	learn: 18.7151848	total: 14.5s	remaining: 3m 11s
800:	learn: 17.8151925	total: 16.4s	remaining: 3m 8s
900:	learn: 17.0132680	total: 18.5s	remaining: 3m 7s
1000:	learn: 16.2768932	total: 20.7s	remaining: 3m 6s
1100:	learn: 15.5921256	total: 22.9s	remaining: 3m 5s
1200:	learn: 14.9548676	total: 25.1s	remaining: 3m 3s
1300:	learn: 14.3561288	total: 27.1s	remaining: 3m
1400:	learn: 13.8012762	total: 29s	remaining: 2m 58s
1500:	learn: 13.2967570	total: 31s	remaining: 2m 55s
1600:	learn: 12.8194887	total: 33.1s	remaining: 2m 53s
1700:	learn: 12.3784821	total: 35.3s	remaining: 2m 52s
1800:	learn: 11.9284535	total: 37.8s	remaining: 2m 52s
1900:	learn: 11.4875890	total: 3

5200:	learn: 4.3694077	total: 1m 58s	remaining: 1m 49s
5300:	learn: 4.2543048	total: 2m	remaining: 1m 46s
5400:	learn: 4.1455871	total: 2m 2s	remaining: 1m 44s
5500:	learn: 4.0420614	total: 2m 4s	remaining: 1m 41s
5600:	learn: 3.9374116	total: 2m 6s	remaining: 1m 39s
5700:	learn: 3.8367175	total: 2m 8s	remaining: 1m 36s
5800:	learn: 3.7420618	total: 2m 10s	remaining: 1m 34s
5900:	learn: 3.6445824	total: 2m 12s	remaining: 1m 32s
6000:	learn: 3.5535028	total: 2m 14s	remaining: 1m 29s
6100:	learn: 3.4629910	total: 2m 16s	remaining: 1m 27s
6200:	learn: 3.3818245	total: 2m 18s	remaining: 1m 24s
6300:	learn: 3.2922524	total: 2m 20s	remaining: 1m 22s
6400:	learn: 3.2095617	total: 2m 22s	remaining: 1m 19s
6500:	learn: 3.1276045	total: 2m 24s	remaining: 1m 17s
6600:	learn: 3.0485055	total: 2m 26s	remaining: 1m 15s
6700:	learn: 2.9734986	total: 2m 28s	remaining: 1m 13s
6800:	learn: 2.8995545	total: 2m 30s	remaining: 1m 10s
6900:	learn: 2.8320165	total: 2m 32s	remaining: 1m 8s
7000:	learn: 2.7629

100:	learn: 28.6871946	total: 2.4s	remaining: 3m 55s
200:	learn: 25.6587851	total: 4.63s	remaining: 3m 45s
300:	learn: 23.6334882	total: 7.06s	remaining: 3m 47s
400:	learn: 21.9715576	total: 9.46s	remaining: 3m 46s
500:	learn: 20.6428894	total: 11.9s	remaining: 3m 44s
600:	learn: 19.5437818	total: 14.5s	remaining: 3m 47s
700:	learn: 18.5931316	total: 17s	remaining: 3m 45s
800:	learn: 17.7423177	total: 19.4s	remaining: 3m 43s
900:	learn: 16.9283436	total: 21.7s	remaining: 3m 39s
1000:	learn: 16.1991734	total: 24s	remaining: 3m 35s
1100:	learn: 15.5549051	total: 26.2s	remaining: 3m 31s
1200:	learn: 14.9455825	total: 28.6s	remaining: 3m 29s
1300:	learn: 14.3562282	total: 31.3s	remaining: 3m 29s
1400:	learn: 13.8059254	total: 33.8s	remaining: 3m 27s
1500:	learn: 13.3138833	total: 36.1s	remaining: 3m 24s
1600:	learn: 12.8203030	total: 38.4s	remaining: 3m 21s
1700:	learn: 12.3632731	total: 40.7s	remaining: 3m 18s
1800:	learn: 11.9241279	total: 42.9s	remaining: 3m 15s
1900:	learn: 11.5018789	

...

4900:	learn: 4.6904816	total: 2m 5s	remaining: 2m 10s
5000:	learn: 4.5673433	total: 2m 8s	remaining: 2m 8s
5100:	learn: 4.4450508	total: 2m 11s	remaining: 2m 6s
5200:	learn: 4.3307274	total: 2m 13s	remaining: 2m 3s
5300:	learn: 4.2217079	total: 2m 16s	remaining: 2m
5400:	learn: 4.1064709	total: 2m 18s	remaining: 1m 58s
5500:	learn: 4.0015286	total: 2m 21s	remaining: 1m 55s
5600:	learn: 3.8934941	total: 2m 23s	remaining: 1m 52s
5700:	learn: 3.7920722	total: 2m 26s	remaining: 1m 50s
5800:	learn: 3.6943882	total: 2m 28s	remaining: 1m 47s
5900:	learn: 3.6026224	total: 2m 32s	remaining: 1m 45s
6000:	learn: 3.5181874	total: 2m 34s	remaining: 1m 43s
6100:	learn: 3.4335860	total: 2m 36s	remaining: 1m 40s
6200:	learn: 3.3475153	total: 2m 39s	remaining: 1m 37s
6300:	learn: 3.2649713	total: 2m 41s	remaining: 1m 35s
6400:	learn: 3.1862664	total: 2m 44s	remaining: 1m 32s
6500:	learn: 3.1058471	total: 2m 47s	remaining: 1m 29s
6600:	learn: 3.0260468	total: 2m 49s	remaining: 1m 27s
6700:	learn: 2.9528

9900:	learn: 1.4027012	total: 4m 9s	remaining: 2.49s
9999:	learn: 1.3697692	total: 4m 11s	remaining: 0us
err:  25.175022241872316
0:	learn: 44.1245067	total: 21ms	remaining: 3m 30s
100:	learn: 28.7800711	total: 2.58s	remaining: 4m 13s
200:	learn: 25.7090433	total: 4.97s	remaining: 4m 2s
300:	learn: 23.6147594	total: 7.19s	remaining: 3m 51s
400:	learn: 21.9910823	total: 9.4s	remaining: 3m 44s
500:	learn: 20.6440555	total: 11.6s	remaining: 3m 40s
600:	learn: 19.5684732	total: 13.8s	remaining: 3m 36s
700:	learn: 18.5793126	total: 16.2s	remaining: 3m 35s
800:	learn: 17.7058948	total: 18.7s	remaining: 3m 35s
900:	learn: 16.9007468	total: 21.2s	remaining: 3m 33s
1000:	learn: 16.1731548	total: 23.5s	remaining: 3m 31s
1100:	learn: 15.5075056	total: 25.8s	remaining: 3m 28s
1200:	learn: 14.8767345	total: 28.1s	remaining: 3m 25s
1300:	learn: 14.2905455	total: 30.3s	remaining: 3m 22s
1400:	learn: 13.7523849	total: 32.9s	remaining: 3m 21s
1500:	learn: 13.2509110	total: 35.4s	remaining: 3m 20s
1600:

4800:	learn: 4.9228889	total: 2m 2s	remaining: 2m 13s
4900:	learn: 4.7975159	total: 2m 5s	remaining: 2m 10s
5000:	learn: 4.6665747	total: 2m 8s	remaining: 2m 7s
5100:	learn: 4.5374148	total: 2m 10s	remaining: 2m 5s
5200:	learn: 4.4165095	total: 2m 12s	remaining: 2m 2s
5300:	learn: 4.3059653	total: 2m 14s	remaining: 1m 59s
5400:	learn: 4.1959073	total: 2m 17s	remaining: 1m 56s
5500:	learn: 4.0923873	total: 2m 19s	remaining: 1m 53s
5600:	learn: 3.9834324	total: 2m 21s	remaining: 1m 51s
5700:	learn: 3.8775818	total: 2m 24s	remaining: 1m 48s
5800:	learn: 3.7764715	total: 2m 26s	remaining: 1m 46s
5900:	learn: 3.6808174	total: 2m 28s	remaining: 1m 43s
6000:	learn: 3.5869897	total: 2m 31s	remaining: 1m 40s
6100:	learn: 3.4956986	total: 2m 33s	remaining: 1m 38s
6200:	learn: 3.4095522	total: 2m 35s	remaining: 1m 35s
6300:	learn: 3.3223282	total: 2m 38s	remaining: 1m 32s
6400:	learn: 3.2387756	total: 2m 40s	remaining: 1m 30s
6500:	learn: 3.1644548	total: 2m 43s	remaining: 1m 27s
6600:	learn: 3.0

9800:	learn: 1.4398993	total: 3m 51s	remaining: 4.71s
9900:	learn: 1.4070323	total: 3m 54s	remaining: 2.34s
9999:	learn: 1.3758886	total: 3m 56s	remaining: 0us
err:  26.980540402062083
0:	learn: 44.2943864	total: 22.8ms	remaining: 3m 48s
100:	learn: 29.0321452	total: 2.23s	remaining: 3m 38s
200:	learn: 25.8679190	total: 4.45s	remaining: 3m 36s
300:	learn: 23.8198904	total: 6.97s	remaining: 3m 44s
400:	learn: 22.1027865	total: 9.45s	remaining: 3m 46s
500:	learn: 20.7459941	total: 11.8s	remaining: 3m 44s
600:	learn: 19.5938130	total: 14s	remaining: 3m 39s
700:	learn: 18.5831048	total: 16.2s	remaining: 3m 35s
800:	learn: 17.6595237	total: 18.4s	remaining: 3m 31s
900:	learn: 16.8717330	total: 20.7s	remaining: 3m 29s
1000:	learn: 16.1172138	total: 23.3s	remaining: 3m 29s
1100:	learn: 15.4405950	total: 25.7s	remaining: 3m 28s
1200:	learn: 14.8304204	total: 28.1s	remaining: 3m 25s
1300:	learn: 14.2488653	total: 30.3s	remaining: 3m 22s
1400:	learn: 13.7130570	total: 32.5s	remaining: 3m 19s
150

4700:	learn: 5.0280509	total: 1m 50s	remaining: 2m 5s
4800:	learn: 4.8938939	total: 1m 53s	remaining: 2m 2s
4900:	learn: 4.7643798	total: 1m 55s	remaining: 1m 59s
5000:	learn: 4.6317040	total: 1m 57s	remaining: 1m 57s
5100:	learn: 4.5146493	total: 2m	remaining: 1m 55s
5200:	learn: 4.4067423	total: 2m 2s	remaining: 1m 53s
5300:	learn: 4.2933810	total: 2m 5s	remaining: 1m 50s
5400:	learn: 4.1831823	total: 2m 7s	remaining: 1m 48s
5500:	learn: 4.0772226	total: 2m 9s	remaining: 1m 45s
5600:	learn: 3.9703091	total: 2m 11s	remaining: 1m 43s
5700:	learn: 3.8709729	total: 2m 14s	remaining: 1m 41s
5800:	learn: 3.7744964	total: 2m 16s	remaining: 1m 39s
5900:	learn: 3.6823058	total: 2m 19s	remaining: 1m 36s
6000:	learn: 3.5902464	total: 2m 21s	remaining: 1m 34s
6100:	learn: 3.4994172	total: 2m 23s	remaining: 1m 32s
6200:	learn: 3.4146617	total: 2m 26s	remaining: 1m 29s
6300:	learn: 3.3329102	total: 2m 29s	remaining: 1m 27s
6400:	learn: 3.2469474	total: 2m 32s	remaining: 1m 25s
6500:	learn: 3.16789

In [15]:
np.mean(testsplit_store)

27.555567874381296

In [16]:
y_pred = np.mean(test_store,0)

In [17]:
y_pred 

array([86.02452536, 44.62568875, 48.64601064, ..., 47.74580758,
       37.82086887, 43.51902271])

In [18]:
sampleSubmission.head()

,Place_ID X Date,target
0,0OS9LVX X 2020-01-02,73.046086
1,0OS9LVX X 2020-01-03,42.552656
2,0OS9LVX X 2020-01-04,47.873672
3,0OS9LVX X 2020-01-05,50.992542
4,0OS9LVX X 2020-01-06,35.338412


In [19]:
sampleSubmission = sampleSubmission.drop(['target'], axis=1)

In [20]:
sampleSubmission['target'] = y_pred

In [21]:
sampleSubmission.to_csv('SampleSubmission.csv', index=False)

In [30]:
#Hyperparameter tuning